In [54]:
from vis.visualization import visualize_saliency, visualize_cam
from keras import Model
from model import unet, aunet
from tqdm import tqdm
from matplotlib import pyplot as plt
import os
import numpy as np
import nibabel as nib

In [2]:
data_root = 'example/data'
model_path = 'example/model'
result_path = 'example/result'
mask_path = 'example/mask'

In [36]:
input_path = os.path.join(data_root, './')
method_path = os.path.join(model_path, 'unet_.hdf5')
method_net = unet(pretrained_weights=method_path)

In [4]:
method_net.get_layer('a6').output

<tf.Tensor 'a6/up_sampling2d_1/ResizeBilinear:0' shape=(?, 20, 28, 1) dtype=float32>

In [37]:
method_net.layers

In [38]:
len(method_net.layers)

42

In [49]:
method_net.get_layer(index=40).output

<tf.Tensor 'conv2d_74/Elu:0' shape=(?, 160, 224, 16) dtype=float32>

In [5]:
layer_net_a6 = Model(inputs=method_net.input, outputs=method_net.get_layer('a6').output)
layer_net_a7 = Model(inputs=method_net.input, outputs=method_net.get_layer('a7').output)
layer_net_a8 = Model(inputs=method_net.input, outputs=method_net.get_layer('a8').output)

In [6]:
# min max scaler
def min_max_scaler(a):
    return (a - np.min(a)) / (np.max(a) - np.min(a))


# crop roi for unet prediction
def crop_roi(img):
    test_img = np.rot90(min_max_scaler(img))
    crop_img = test_img[130:130 + 160, 50:50 + 224]
    return crop_img

In [7]:
NII_fileList = []
for dirName, subdirList, fileList in os.walk(input_path):
    for filename in fileList:
        if ".nii" in filename.lower():
            # print filename
            NII_fileList.append(os.path.join(dirName, filename))

In [61]:
for file in tqdm(NII_fileList, desc='Detect in {}'.format(os.path.abspath(input_path))):
    filename = os.path.basename(file)
    #_, _ = detect_file(file, model, layer_net)
    data = nib.load(file)
    
    width, height, frame_num = data.shape
    matrix = data.get_data()
    #unet_matrix = bet_unet(matrix, layer_net, threshold=0.2)
    img_batch = np.transpose(matrix, (2, 0, 1))
    #mask_batch = unet_batch_predict(img_batch, layer_net, threshold=0.2)
    batch, h, w = img_batch.shape
    crop_batch = []
    for img in img_batch:
        crop_img = crop_roi(img)
        crop_batch.append(crop_img)
    crop_batch = np.array(crop_batch)
    crop_batch = np.reshape(crop_batch, crop_batch.shape + (1,))
    
saliency_map = visualize_saliency(model=method_net, layer_idx=41, seed_input=crop_batch[0], filter_indices=0)

Detect in F:\GitHub\DICOM\example\data: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:00<00:00, 20.42it/s]


InvalidArgumentError: input_3:0 is both fed and fetched.

In [60]:
crop_batch.shape

(30, 160, 224, 1)

In [27]:
from vis.utils import utils

In [28]:
layer_idx = utils.find_layer_idx(method_net, 'a6')

In [29]:
layer_idx

21

In [53]:
method_net.output

<tf.Tensor 'conv2d_75/Sigmoid:0' shape=(?, 160, 224, 1) dtype=float32>